## Collecting image and subscriber data using youtube api

1. 기존 데이터 파일(titles_preprocessed.csv)에서 video_id, channel_id를 불러온다.
2. channel api에서 구독자 수를 불러온다.
3. video api에서 thumbnail 이미지를 불러온다.
4. 불러온 정보를 담음 새로운 csv 파일을 생성한다.

In [1]:
import pandas as pd
from googleapiclient.discovery import build
import config

In [2]:
youtube = build('youtube', 'v3', developerKey = config.API_KEY)

### 1. 기존 데이터 파일(titles_preprocessed.csv)에서 video_id, channel_id를 불러온다.

In [19]:
#첫 수집 시
data_original = pd.read_csv("./titles_preprocessed.csv")
df = pd.DataFrame(data_original)
df['subscriber_count']=-1
df['thumbnail']=-1
df

,title,video_id,channel_name,channel_id,publish_time,views,preprocessed_title,subscriber_count,thumbnail
0,만 19세 '연봉 1억' 친구와 레스토랑 브이로그,8e7GcsDEKEE,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-04-01T11:47:06Z,30038.0,만 19세 연봉 1억 친구와 레스토랑 브이로그,-2,-2
1,BBQ 신메뉴 '체고치' 순살 먹어봄,FYoDak3VfF4,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-29T10:41:55Z,29102.0,bbq 신메뉴 체고치 순살 먹어봄,-2,-2
2,구찌 라이톤이랑 시계 후기,FuTTOzwWo58,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-26T06:54:42Z,25022.0,구찌 라이톤이랑 시계 후기,-2,-2
3,엽기 로제떡볶이랑 허니 콤보 혼내줌,-48XAC_GDjk,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-23T11:58:49Z,49551.0,엽기 로제떡볶이랑 허니 콤보 혼내줌,-2,-2
4,어림도 없지 바로 마라탕 먹어버림,mhehORE9WNA,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-20T11:30:18Z,74348.0,어림도 없지 바로 마라탕 먹어버림,-2,-2
...,...,...,...,...,...,...,...,...,...
101159,'현실남매' 먹방하러 갔다가 진짜 싸웠습니다.,K-sH6E9YeTw,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-21T12:14:50Z,137922.0,현실남매 먹방하러 갔다가 진짜 싸웠습니다,-2,-2
101160,[직업체험] DJ예나 일일 매니저 (월디페/World DJ Festival),ucEs84amVm8,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-14T17:10:40Z,153005.0,직업체험 dj예나 일일 매니저 월디페 world dj festival,-2,-2
101161,저의 슈퍼모델 친구들을 소개합니다.,jSK3H-dFfE0,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-10T09:08:41Z,151071.0,저의 슈퍼모델 친구들을 소개합니다,-2,-2
101162,'현실남매' 우리만 이런건가요? 남들이 이해 못하는 현실남매,OEWk-sZG5j0,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-02T09:05:01Z,177515.0,현실남매 우리만 이런건가요 ? 남들이 이해 못하는 현실남매,-2,-2


In [77]:
#이어 수집할 때
data_original = pd.read_csv("./data_including_subscriber_and_thumbnail.csv")
df = pd.DataFrame(data_original)
df

,title,video_id,channel_name,channel_id,publish_time,views,preprocessed_title,subscriber_count,thumbnail
0,만 19세 '연봉 1억' 친구와 레스토랑 브이로그,8e7GcsDEKEE,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-04-01T11:47:06Z,30038.0,만 19세 연봉 1억 친구와 레스토랑 브이로그,98400,-1
1,BBQ 신메뉴 '체고치' 순살 먹어봄,FYoDak3VfF4,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-29T10:41:55Z,29102.0,bbq 신메뉴 체고치 순살 먹어봄,98400,-1
2,구찌 라이톤이랑 시계 후기,FuTTOzwWo58,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-26T06:54:42Z,25022.0,구찌 라이톤이랑 시계 후기,98400,-1
3,엽기 로제떡볶이랑 허니 콤보 혼내줌,-48XAC_GDjk,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-23T11:58:49Z,49551.0,엽기 로제떡볶이랑 허니 콤보 혼내줌,98400,-1
4,어림도 없지 바로 마라탕 먹어버림,mhehORE9WNA,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-20T11:30:18Z,74348.0,어림도 없지 바로 마라탕 먹어버림,98400,-1
...,...,...,...,...,...,...,...,...,...
101159,'현실남매' 먹방하러 갔다가 진짜 싸웠습니다.,K-sH6E9YeTw,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-21T12:14:50Z,137922.0,현실남매 먹방하러 갔다가 진짜 싸웠습니다,112000,-1
101160,[직업체험] DJ예나 일일 매니저 (월디페/World DJ Festival),ucEs84amVm8,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-14T17:10:40Z,153005.0,직업체험 dj예나 일일 매니저 월디페 world dj festival,112000,-1
101161,저의 슈퍼모델 친구들을 소개합니다.,jSK3H-dFfE0,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-10T09:08:41Z,151071.0,저의 슈퍼모델 친구들을 소개합니다,112000,-1
101162,'현실남매' 우리만 이런건가요? 남들이 이해 못하는 현실남매,OEWk-sZG5j0,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-02T09:05:01Z,177515.0,현실남매 우리만 이런건가요 ? 남들이 이해 못하는 현실남매,112000,-1


### 2. channel api에서 구독자 수를 불러온다.

In [86]:
subscriber_index = -1
for index in range(len(df)):
    if(df['subscriber_count'][index]==-1):
        subscriber_index = index
        break
print(subscriber_index)

-1


In [79]:
def get_youtube_subscriber_count():
    for index in range(subscriber_index, len(df)):
        if(index!=0 and df.loc[index, 'channel_id']==df.loc[index-1, 'channel_id']):
            subscriber_count = df.loc[index-1,'subscriber_count']
        else: #새로운 채널
            content = youtube.channels().list(
                id = df.loc[index, 'channel_id'], 
                part = 'statistics'
            ).execute()
            
            if(content['pageInfo']['totalResults']==0): #결과 없음
                subscriber_count = -3
            elif(content['items'][0]['statistics']['hiddenSubscriberCount']==True):
                subscriber_count = -2
            else:
                subscriber_count = content['items'][0]['statistics']['subscriberCount']

        df.loc[index,'subscriber_count']= subscriber_count

In [81]:
get_youtube_subscriber_count()

In [83]:
df.to_csv("data_including_subscriber_and_thumbnail.csv", header=True, index=False)
df

,title,video_id,channel_name,channel_id,publish_time,views,preprocessed_title,subscriber_count,thumbnail
0,만 19세 '연봉 1억' 친구와 레스토랑 브이로그,8e7GcsDEKEE,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-04-01T11:47:06Z,30038.0,만 19세 연봉 1억 친구와 레스토랑 브이로그,98400,-1
1,BBQ 신메뉴 '체고치' 순살 먹어봄,FYoDak3VfF4,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-29T10:41:55Z,29102.0,bbq 신메뉴 체고치 순살 먹어봄,98400,-1
2,구찌 라이톤이랑 시계 후기,FuTTOzwWo58,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-26T06:54:42Z,25022.0,구찌 라이톤이랑 시계 후기,98400,-1
3,엽기 로제떡볶이랑 허니 콤보 혼내줌,-48XAC_GDjk,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-23T11:58:49Z,49551.0,엽기 로제떡볶이랑 허니 콤보 혼내줌,98400,-1
4,어림도 없지 바로 마라탕 먹어버림,mhehORE9WNA,희준,UCVa3PvcBT187XSV6pFAZQwg,2021-03-20T11:30:18Z,74348.0,어림도 없지 바로 마라탕 먹어버림,98400,-1
...,...,...,...,...,...,...,...,...,...
101159,'현실남매' 먹방하러 갔다가 진짜 싸웠습니다.,K-sH6E9YeTw,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-21T12:14:50Z,137922.0,현실남매 먹방하러 갔다가 진짜 싸웠습니다,112000,-1
101160,[직업체험] DJ예나 일일 매니저 (월디페/World DJ Festival),ucEs84amVm8,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-14T17:10:40Z,153005.0,직업체험 dj예나 일일 매니저 월디페 world dj festival,112000,-1
101161,저의 슈퍼모델 친구들을 소개합니다.,jSK3H-dFfE0,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-10T09:08:41Z,151071.0,저의 슈퍼모델 친구들을 소개합니다,112000,-1
101162,'현실남매' 우리만 이런건가요? 남들이 이해 못하는 현실남매,OEWk-sZG5j0,최은경tv,UCQV67-DsnpfkjPTjM_ITtSA,2019-06-02T09:05:01Z,177515.0,현실남매 우리만 이런건가요 ? 남들이 이해 못하는 현실남매,112000,-1


### 3. video api에서 thumbnail 이미지를 불러온다.

In [87]:
thumbnail_index = -1
for index in range(len(df)):
    if(df['thumbnail'][index]==-1):
        thumbnail_index = index
        break
print(thumbnail_index)

0


In [ ]:
#아직 x
def get_youtube_thumbnails():
    thumbnails = []
    for video_id in video_ids[1]:
        content = youtube.videos().list(
            id = video_id,
            part = 'snippet'
        ).execute()
        
        search_response = content['items'][0]['snippet']
        
        thumbnail = search_response.['thumbnails']
        thumbnails.append(thumbnail)
    return thumbnails
def get_youtube_subscriber_count():
    for index in range(290, 320):#range(subscriber_index, len(df)):
        if(index!=0 and df.loc[index, 'channel_id']==df.loc[index-1, 'channel_id']):
            subscriber_count = df.loc[index-1,'subscriber_count']
        else: #새로운 채널
            content = youtube.channels().list(
                id = df.loc[index, 'channel_id'], 
                part = 'statistics'
            ).execute()
            
            if(content['pageInfo']['totalResults']==0): #결과 없음
                subscriber_count = -3
            elif(content['items'][0]['statistics']['hiddenSubscriberCount']==True):
                subscriber_count = -2
            else:
                subscriber_count = content['items'][0]['statistics']['subscriberCount']

        df.loc[index,'subscriber_count']= subscriber_count

In [ ]:
thumbnails = get_youtube_thumbnails()
print(video_ids)
print(len(video_ids), 'to', len(thumbnails))

### 4. 불러온 정보를 담음 새로운 csv 파일을 생성한다.

In [ ]:
df = pd.DataFrame(data_original)
df['subscriber_count'] = subscriber_counts
df.to_csv("data_including_subscriber_and_thumbnail.csv", header=True, index=False)
df